In [ ]:
#import ALL THE THINGS
from ema_workbench import (Model, 
                           RealParameter, CategoricalParameter, IntegerParameter, Constant, 
                           ScalarOutcome, 
                           perform_experiments, MultiprocessingEvaluator, 
                           ema_logging, save_results)

from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.evaluators import LHS, MC, FF, PFF, MORRIS, SOBOL, FAST

ema_logging.log_to_stderr(ema_logging.INFO)

# Configuring the Model
Below are the recommended uncertanties and constants to run the basic model for initial data collection purposes. 

|Uncertainties|Constants|
|--|--|
|<table> <tr><th>Uncertainty</th><th>Range</th><th>Default value</th></tr><tr><td>Average incubation time</td><td>0.5 – 14</td><td>2</td></tr><tr><td>Average infectious period</td><td>7 – 21</td><td>14</td></tr><tr><td>Basic reproduction number</td><td>0.5 – 5</td><td>2</td></tr><tr><td>Case fatality rate	</td><td>0.01 – 0.5</td><td>0.01</td></tr><tr><td>Switch SEIR or SIR</td><td>{1, 0}</td><td>0</td></tr> </table>| <table> <tr><th>Constant</th><th>Value</th></tr><tr><td>Initial deceased population</td><td>0</td></tr><tr><td>Initial exposed population</td><td>0</td></tr><tr><td>Initial infectious population</td><td>1</td></tr><tr><td>Initial recovered population</td><td>0</td></tr><tr><td>Initial susceptible population</td><td>100000</td></tr> </table>|

In [ ]:
#We can define common uncertainties and outcomes for each model:
uncertainties = [RealParameter('Average incubation time', 0.5, 14),
                 RealParameter('Average infectious period', 7, 21),
                 RealParameter('Basic reproduction number', 0.5, 5),
                 RealParameter('Case fatality rate', 0.01, 0.5)] 

outcomes = [ScalarOutcome('TIME'),
            ScalarOutcome('Deceased population'),
            ScalarOutcome('Infectious population'),
            ScalarOutcome('Recovered population'),
            ScalarOutcome('Susceptible population'),
            ScalarOutcome('Exposed population'),
            ScalarOutcome('Switch SEIR or SIR')]

# Ensure the proper constant values based on information above
constants = [Constant('Initial deceased population', 0),
             Constant('Initial exposed population', 0),
             Constant('Initial infectious population', 1),
             Constant('Initial recovered population', 0),
             Constant('Initial susceptible population', 100000)]

In [ ]:
models = [{
            'file':'models/01_SEIR_SIR_Original_RealSwitch.mdl',
            'name':'real', 
            'uncertainties': uncertainties,
            'outcomes': outcomes, 
            'constants': constants
         }, {
            'file':'models/01_SEIR_SIR_Original.mdl',
            'name':'category', 
            'uncertainties': uncertainties,
            'outcomes': outcomes, 
            'constants': constants
         }]

models[0]['uncertainties'] = models[0]['uncertainties'] + [RealParameter('SwitchInput', 0.0001, 1)] #avoid 0 in range
models[1]['uncertainties'] = models[1]['uncertainties'] + [CategoricalParameter('Switch SEIR or SIR', [1,0])]

In [ ]:
#requirement for sobol to run properly
sobol_mult = (2*len(models[0]['uncertainties'])+2) # 5 = number of uncertainties

samplings = [{'method':LHS,'name':'LHS'},{'method':MC,'name':'MC'}, 
             {'method':PFF,'name':'PFF'}]

num_experiments = [100*sobol_mult,1000*sobol_mult]

In [ ]:
for model in models: 
    for sampling in samplings: 
        for num_exp in num_experiments: 

            mdl = PysdModel(model['name'], mdl_file=model['file'])

            mdl.uncertainties = model['uncertainties']
            mdl.outcomes = model['outcomes']
            mdl.constants = model['constants']
            mdl.time_horizon = 360

            # Single processing execution for PCs
            # results = perform_experiments(mdl, num_exp, uncertainty_sampling=sampling['method'])

            # Multiprocessing evaluator for Macs
            with MultiprocessingEvaluator(mdl) as evaluator:
                results = evaluator.perform_experiments(num_exp, uncertainty_sampling=sampling['method'])

            results_name = './data/' + model['name']+'_'+sampling['name']+'_'+str(num_exp)+'.tar.gz'
            save_results(results, results_name)

# Vensim Generator

In [ ]:
from ema_workbench.connectors.vensim import VensimModel

uncertainties = [RealParameter('Average incubation time', 0.5, 14),
                 RealParameter('Average infectious period', 7, 21),
                 RealParameter('Basic reproduction number', 0.5, 5),
                 RealParameter('Case fatality rate', 0.01, 0.5),
                 IntegerParameter('Switch SEIR or SIR', 0,1), 
                 
                 RealParameter('Base Transmission Coefficient Social Distance III', 0.7, 0.8),
                 RealParameter('Base Societal Stress Coefficient Social Distance III', 0.5, 0.9),
                 RealParameter('Base Effort Coefficient Social Distance III', 4, 8),
                 RealParameter('Implementation Time Social Distance III', 2, 4),

                 RealParameter('Base Transmission Coefficient Social Distance II', 0.89, 0.96),
                 RealParameter('Base Societal Stress Coefficient Social Distance II', 0.3, 0.8),
                 RealParameter('Base Effort Coefficient Social Distance II', 3, 6),
                 RealParameter('Implementation Time Social Distance II', 1, 3),

                 RealParameter('Base Transmission Coefficient Social Distance I', 0.92, 0.97),
                 RealParameter('Base Societal Stress Coefficient Social Distance I', 0.2, 0.3),
                 RealParameter('Base Effort Coefficient Social Distance I', 2, 3),
                 RealParameter('Implementation Time Social Distance I', 1, 3),

                 RealParameter('Base Transmission Coefficient Public Info', 0.97, 0.99),
                 RealParameter('Base Societal Stress Coefficient Public Info', 0.01, 0.1),
                 RealParameter('Base Effort Coefficient Public Info', 1, 2),
                 RealParameter('Implementation Time Public Info', 2, 5),

                 RealParameter('Base Transmission Coefficient Public Edu', 0.97, 0.99),
                 RealParameter('Base Societal Stress Coefficient Public Edu', 0.01, 0.1),
                 RealParameter('Base Effort Coefficient Public Edu', 1, 2),
                 RealParameter('Implementation Time Public Edu', 2, 5),

                 RealParameter('Base Transmission Coefficient Facemasks', 0.93, 0.97),
                 RealParameter('Base Societal Stress Coefficient Facemasks', 0.1, 0.15),
                 RealParameter('Base Effort Coefficient Facemasks', 2, 3),
                 RealParameter('Implementation Time Facemasks', 5, 10),

                 RealParameter('Base Transmission Coefficient Tracing', 0.87, 0.93),
                 RealParameter('Base Societal Stress Coefficient Tracing', 0.1, 0.3),
                 RealParameter('Base Effort Coefficient Tracing', 3, 7),
                 RealParameter('Implementation Time Tracing', 2, 10),

                 RealParameter('Base Transmission Coefficient Case Isolation', 0.73, 0.87),
                 RealParameter('Base Recovery Coefficient Case Isolation', 0.92, 0.97),
                 RealParameter('Base Societal Stress Coefficient Case Isolation', 0.3, 0.6),
                 RealParameter('Base Effort Coefficient Case Isolation', 5, 9),
                 RealParameter('Implementation Time Case Isolation', 14, 28),

                 RealParameter('Base Societal Stress Coefficient Involuntary Immunization', 0.5, 0.8),
                 RealParameter('Base Effort Coefficient Involuntary Immunization', 5, 10),
                 RealParameter('Implementation Time Involuntary Immunization', 30, 180),

                 RealParameter('Base Societal Stress Coefficient Voluntary Immunization', 0.1, 0.3),
                 RealParameter('Base Effort Coefficient Voluntary Immunization', 2, 6),
                 RealParameter('Implementation Time Voluntary Immunization', 30, 180),

                 RealParameter('Base Transmission Coefficient Medical Care I', 0.9, 0.97),
                 RealParameter('Base Recovery Coefficient Medical Care I', 0.97, 1),
                 RealParameter('Base Societal Stress Coefficient Medical Care I', 0.05, 0.1),
                 RealParameter('Base Effort Coefficient Medical Care I', 1, 4),
                 RealParameter('Implementation Time Medical Care I', 5, 20),

                 RealParameter('Base Transmission Coefficient Medical Care II', 0.83, 0.9),
                 RealParameter('Base Recovery Coefficient Medical Care II', 0.7, 0.98),
                 RealParameter('Base Societal Stress Coefficient Medical Care II', 0.05, 0.1),
                 RealParameter('Base Effort Coefficient Medical Care II', 4, 8),
                 RealParameter('Implementation Time Medical Care II', 20, 40),
                ]

outcomes = [ScalarOutcome('Deceased population'),
            ScalarOutcome('Infectious population'),
            ScalarOutcome('Recovered population'),
            ScalarOutcome('Susceptible population'),
            ScalarOutcome('Exposed population'),
            ScalarOutcome('Stock Total Effort'),
            ScalarOutcome('Stock Societal Stress')]

# Ensure the proper constant values based on information above
constants = [Constant('Initial deceased population', 0),
             Constant('Initial exposed population', 0),
             Constant('Initial infectious population', 1),
             Constant('Initial recovered population', 0),
             Constant('Initial susceptible population', 100000), 
            
             Constant('Duration Social Distance III', 360),
             Constant('Duration Social Distance II', 360),
             Constant('Duration Social Distance I', 360),
             Constant('Duration Public Info', 360),
             Constant('Duration Public Edu', 360),
             Constant('Duration Facemasks', 360),
             Constant('Duration Tracing', 360),
             Constant('Duration Case Isolation', 360),
             Constant('Duration Involuntary Immunization', 360),
             Constant('Duration Voluntary Immunization', 360),
             Constant('Duration Medical Care I', 360),
             Constant('Duration Medical Care II', 360),
            ]

mdl = VensimModel('Model', wd=r'./models', model_file=r'04_SEIR_SIR_FullPolicy.vpm')
mdl.uncertainties = uncertainties
mdl.outcomes = outcomes
mdl.constants = constants
mdl.time_horizon = 360

results = perform_experiments(mdl, 3000, uncertainty_sampling=LHS)

results_name = './data/FullModel_LHS_3000.tar.gz'
save_results(results, results_name)